### Recipes 5k

* [Dataset](http://www.ub.edu/cvub/recipes5k/)

* [Original Paper](https://www.researchgate.net/publication/318729535_Food_Ingredients_Recognition_through_Multi-label_Learning)

In [1]:
%cd ..

/home/datascience/Data Fusion


### Setup Environment:

In [2]:
import os
import pandas as pd

from src.classifiers import preprocess_data, process_labels,split_data

from src.classifiers import VQADataset
from torch.utils.data import DataLoader

from src.classifiers_cpu_metrics import calculate_memory

In [3]:
PATH = 'Embeddings_vlm/Recipes5k/'
FILE = 'embeddings_clip.csv'

FILE_PATH = os.path.join(PATH, FILE)

## Get data

In [4]:
df = pd.read_csv(FILE_PATH)
df.drop(columns=['image'], inplace=True)
df.head()

,class,split,ingredients,image_embedding_0,image_embedding_1,image_embedding_2,image_embedding_3,image_embedding_4,image_embedding_5,image_embedding_6,...,text_embedding_502,text_embedding_503,text_embedding_504,text_embedding_505,text_embedding_506,text_embedding_507,text_embedding_508,text_embedding_509,text_embedding_510,text_embedding_511
0,onion_rings,val,"yellow onion,flour,baking powder,seasoning sal...",-0.019941,0.046292,0.028163,0.013430,0.014544,0.018990,0.053650,...,-0.039769,0.010437,0.027736,0.016436,-0.014798,-0.043962,0.084459,0.100879,-0.040810,-0.026944
1,onion_rings,train,"white onion,panko,cornmeal,ground paprika,onio...",0.020726,0.047324,-0.009278,0.030066,0.009650,-0.022774,-0.009808,...,-0.028915,0.007126,0.033126,0.014813,0.006720,-0.033571,0.069636,0.070107,-0.047277,-0.035330
2,onion_rings,train,"yellow onion,all-purpose flour,baking powder,l...",-0.003352,0.036595,-0.010682,-0.004273,0.002105,0.032441,0.023189,...,0.004127,0.042316,0.007637,0.010198,-0.027701,-0.021962,0.090450,0.056628,-0.020075,-0.020748
3,onion_rings,train,"oil,pancake mix,spanish onion",-0.038810,0.060136,0.016056,0.013183,-0.003867,0.000385,0.018509,...,-0.009147,0.037166,0.001721,-0.013780,-0.009100,-0.031707,0.064553,0.031182,0.002685,-0.022401
4,onion_rings,train,"peanut oil,sweet onion,flour,eggs,celery salt,...",-0.012332,0.064609,-0.033546,-0.007815,0.015096,0.002560,0.014974,...,-0.009159,0.021923,0.001067,0.040090,-0.018709,-0.020262,0.047931,0.071678,-0.000059,-0.041866


## Data Perparation

In [5]:
# Split the data
train_df, test_df = split_data(df)

# Select features and labels vectors
text_columns = [column for column in df.columns if 'text' in column] #[f'text_{i}']
image_columns = [column for column in df.columns if 'image' in column] #[f'image_{i}']
label_columns = 'class'


# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)


train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Train Shape: (3409, 1027)
Test Shape: (783, 1027)


### Models

In [6]:
text_input_size = len(text_columns)
image_input_size = len(image_columns)
output_size = len(mlb.classes_)
multilabel = False

In [7]:
calculate_memory(train_loader, test_loader, text_input_size, image_input_size, output_size)

Early fusion:
Average Memory per Batch in Train: 0.27 MB
Total Memory Usage per Epoch Train: 14.63 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 0.06 MB
Total Memory Usage per Epoch Test: 0.81 MB (excluding model parameters)
Model: 
Model Memory Usage: 0.55 MB

Late fusion:
Average Memory per Batch in Train: 0.27 MB
Total Memory Usage per Epoch Train: 14.63 MB (excluding model parameters)
Test:
Average Memory per Batch in Test: 0.06 MB
Total Memory Usage per Epoch Test: 0.81 MB (excluding model parameters)
Model: 
Model Memory Usage: 0.30 MB
